# Data source

Data for download: https://storage.googleapis.com/download.tensorflow.org/example_images/flower_photos.tgz

Resources used as reference:
1) Training the image classifier to recognize different species of flowers:
https://www.kaggle.com/dtosidis/flower-classifier-tensorflow
  
2) Loading and preprocessing an image dataset
https://www.tensorflow.org/tutorials/load_data/images

3) Data augmentation
https://www.tensorflow.org/tutorials/images/data_augmentation

4) Image classification
https://www.tensorflow.org/tutorials/images/classification


In [29]:
!pip install keras

  Using cached Keras-2.4.3-py2.py3-none-any.whl (36 kB)


In [47]:
# Dependencies
import matplotlib.pyplot as plt
%matplotlib inline

import os
import numpy as np
import tensorflow as tf

import PIL
import PIL.Image

import pathlib

os.environ['KMP_DUPLICATE_LIB_OK']='True'

from tensorflow import keras
from tensorflow.keras.preprocessing import image
import tensorflow_hub as hub

from keras.datasets import mnist
from keras.layers.core import Dense, Dropout, Activation
from keras.models import Sequential

from tensorflow.keras.applications.vgg19 import (
    VGG19, 
    preprocess_input, 
    decode_predictions
)

#New method for test train split

from sklearn.model_selection import train_test_split



In [48]:
!pwd

/c/ShankersDocs/EDUCATION/RICE_Bootcamp_DataAnalytics/FinalProject_Img_Recognition_Flowers/Final_RICEproject_ImageRecognition_flowers


### Total images in the dataset

In [49]:
data_dir = 'flower_photos'
data_dir = pathlib.Path(data_dir)
image_count = len(list(data_dir.glob('*/*.jpg')))
# print(image_count)

## Image Classification
https://www.tensorflow.org/tutorials/images/classification?hl=zh-tw

In [50]:

batch_size = 32
img_height = 180
img_width = 180

## Generating datasets
https://keras.io/examples/vision/image_classification_from_scratch/

### Generating a training dataset

In [51]:
# When the subset below is defined as "training" the 0.2 validation split takes 80% of the data as the training set

train_ds = tf.keras.preprocessing.image_dataset_from_directory(
  'flower_photos',
  validation_split=0.2,
  subset="training",
  seed=123,
  image_size=(img_height, img_width),
  batch_size=batch_size)

Found 3670 files belonging to 5 classes.
Using 2936 files for training.


### Generating a validation dataset

In [52]:
# When the subset below is defined as "validation" the 0.1 validation split takes 10% of the data as the validation set

val_ds = tf.keras.preprocessing.image_dataset_from_directory(
  'flower_photos',
  validation_split=0.1,
  subset="validation",
  seed=123,
  image_size=(img_height, img_width),
  batch_size=batch_size)

Found 3670 files belonging to 5 classes.
Using 367 files for validation.


### Generating a test dataset

In [53]:
# When the subset below is defined as "validation" the 0.1 validation split takes 10% of the data as the test set

test_ds = tf.keras.preprocessing.image_dataset_from_directory(
  'flower_photos',
  validation_split=0.1,
  subset="validation",
  seed=123,
  image_size=(img_height, img_width),
  batch_size=batch_size)

Found 3670 files belonging to 5 classes.
Using 367 files for validation.


### Class names

In [54]:
class_names = train_ds.class_names
print(class_names)

['daisy', 'dandelion', 'roses', 'sunflowers', 'tulips']


### Rescaling the data
https://www.tensorflow.org/api_docs/python/tf/keras/layers/experimental/preprocessing/Rescaling


In [55]:
from tensorflow.keras import layers

normalization_layer = tf.keras.layers.experimental.preprocessing.Rescaling(1./255)

### Normalizing the data (trainign and validation datasets)

In [56]:
normalized_train_ds = train_ds.map(lambda x, y: (normalization_layer(x), y))
normalized_val_ds =  val_ds.map(lambda x, y: (normalization_layer(x), y))
image_batch, labels_batch = next(iter(normalized_train_ds))
first_image = image_batch[0]
# Notice the pixels values are now in `[0,1]`.
print(np.min(first_image), np.max(first_image)) 

0.0 1.0


### Autotune is done to cache data and make processing and resource mgmt more effieicient

In [57]:
AUTOTUNE = tf.data.experimental.AUTOTUNE

train_ds = train_ds.cache().prefetch(buffer_size=AUTOTUNE)
val_ds = val_ds.cache().prefetch(buffer_size=AUTOTUNE)

In [58]:
normalized_train_ds = normalized_train_ds.cache().prefetch(buffer_size=AUTOTUNE)
normalized_val_ds = normalized_val_ds.cache().prefetch(buffer_size=AUTOTUNE)

In [59]:
# num_classes = 5
num_classes = len(class_names)
num_classes

5

## Model 1 (Sequential Model)
https://www.tensorflow.org/guide/keras/sequential_model

In [ ]:
model = tf.keras.models.Sequential([
  tf.keras.layers.Flatten(input_shape=(img_height, img_width, 3)),
  tf.keras.layers.Dense(128,activation='relu'),
  tf.keras.layers.Dense(num_classes, activation='softmax')
])


In [ ]:
model.compile(
    loss='sparse_categorical_crossentropy',
    optimizer=tf.keras.optimizers.Adam(0.001),
    metrics=['accuracy'],
)

In [ ]:
model.fit(
    train_ds,
    validation_data = val_ds, 
    epochs=16
)

## Model 1 (Sequential Model) Epoch sensitivity

In [ ]:
model = tf.keras.models.Sequential([
  tf.keras.layers.Flatten(input_shape=(img_height, img_width, 3)),
  tf.keras.layers.Dense(128,activation='relu'),
  tf.keras.layers.Dense(num_classes, activation='softmax')
])


In [ ]:
model.compile(
    loss='sparse_categorical_crossentropy',
    optimizer=tf.keras.optimizers.Adam(0.001),
    metrics=['accuracy'],
)

In [ ]:
model.fit(
    train_ds,
    validation_data = val_ds, 
    epochs=6
)

## Model 1a (Sequential Model) With Normalized data

In [19]:
model_1a = tf.keras.models.Sequential([
  tf.keras.layers.Flatten(input_shape=(img_height, img_width, 3)),
  tf.keras.layers.Dense(128,activation='relu'),
  tf.keras.layers.Dense(num_classes, activation='softmax')
])


In [20]:
model_1a.compile(
    loss='sparse_categorical_crossentropy',
    optimizer=tf.keras.optimizers.Adam(0.001),
    metrics=['accuracy'],
)

In [42]:
model_1a.fit(
    normalized_train_ds,
    validation_data = normalized_val_ds, 
    epochs=6
)

Epoch 1/6
92/92 [==============================] - 6s 67ms/step - loss: 9.3329 - accuracy: 0.2548 - val_loss: 1.5924 - val_accuracy: 0.2834
Epoch 2/6
92/92 [==============================] - 6s 66ms/step - loss: 1.5261 - accuracy: 0.3392 - val_loss: 1.5553 - val_accuracy: 0.3406
Epoch 3/6
92/92 [==============================] - 6s 65ms/step - loss: 1.5153 - accuracy: 0.3236 - val_loss: 1.5620 - val_accuracy: 0.3651
Epoch 4/6
92/92 [==============================] - 6s 65ms/step - loss: 1.4879 - accuracy: 0.3304 - val_loss: 1.5920 - val_accuracy: 0.4033
Epoch 5/6
92/92 [==============================] - 6s 65ms/step - loss: 1.4419 - accuracy: 0.3675 - val_loss: 1.5167 - val_accuracy: 0.4142
Epoch 6/6
92/92 [==============================] - 6s 68ms/step - loss: 1.4356 - accuracy: 0.3658 - val_loss: 1.5409 - val_accuracy: 0.4142


In [ ]:
model_1a.fit(
    normalized_train_ds,
    validation_data = normalized_val_ds, 
    epochs=50
)

## Model 2 

In [21]:
model_2 = tf.keras.Sequential([
  layers.experimental.preprocessing.Rescaling(1./255),
  layers.Conv2D(32, 3, activation='relu'),
  layers.MaxPooling2D(),
  layers.Conv2D(32, 3, activation='relu'),
  layers.MaxPooling2D(),
  layers.Conv2D(32, 3, activation='relu'),
  layers.MaxPooling2D(),
  layers.Flatten(),
  layers.Dense(128, activation='relu'),
  layers.Dense(num_classes)
])

In [22]:
model_2.compile(
  optimizer='adam',
  loss=tf.losses.SparseCategoricalCrossentropy(from_logits=True),
  metrics=['accuracy'])

In [30]:
model_2.fit(
  train_ds,
  validation_data=val_ds,
  epochs=1
)

92/92 [==============================] - 60s 657ms/step - loss: 1.3110 - accuracy: 0.4138 - val_loss: 1.1323 - val_accuracy: 0.5722


In [23]:
model_2.fit(
  train_ds,
  validation_data=val_ds,
  epochs=100
)

Epoch 1/100
92/92 [==============================] - 72s 786ms/step - loss: 1.3423 - accuracy: 0.4111 - val_loss: 1.0597 - val_accuracy: 0.5858
Epoch 2/100
92/92 [==============================] - 71s 771ms/step - loss: 1.0217 - accuracy: 0.6005 - val_loss: 1.0120 - val_accuracy: 0.6076
Epoch 3/100
92/92 [==============================] - 71s 772ms/step - loss: 0.8735 - accuracy: 0.6655 - val_loss: 0.9446 - val_accuracy: 0.6403
Epoch 4/100
92/92 [==============================] - 71s 772ms/step - loss: 0.6929 - accuracy: 0.7398 - val_loss: 0.9795 - val_accuracy: 0.6403
Epoch 5/100
92/92 [==============================] - 143s 2s/step - loss: 0.5100 - accuracy: 0.8191 - val_loss: 1.0341 - val_accuracy: 0.6676
Epoch 6/100
92/92 [==============================] - 157s 2s/step - loss: 0.4137 - accuracy: 0.8559 - val_loss: 1.2943 - val_accuracy: 0.6185
Epoch 7/100
92/92 [==============================] - 158s 2s/step - loss: 0.3521 - accuracy: 0.8740 - val_loss: 1.1599 - val_accuracy: 0.654

## TRY MODEL WITH DIFFERENT LOSS PARAMETER

In [62]:
model_2 = tf.keras.Sequential([
  layers.experimental.preprocessing.Rescaling(1./255),
  layers.Conv2D(32, 3, activation='relu'),
  layers.MaxPooling2D(),
  layers.Conv2D(32, 3, activation='relu'),
  layers.MaxPooling2D(),
  layers.Conv2D(32, 3, activation='relu'),
  layers.MaxPooling2D(),
  layers.Flatten(),
  layers.Dense(128, activation='relu'),
  layers.Dense(num_classes)
])

In [67]:
model_2.compile(
  optimizer='adam',
  loss = tf.keras.losses.categorical_crossentropy(class_names,class_names),
  metrics=['accuracy'])

InvalidArgumentError: Value for attr 'T' of string is not in the list of allowed values: float, double, int32, uint8, int16, int8, complex64, int64, qint8, quint8, qint32, bfloat16, uint16, complex128, half, uint32, uint64
	; NodeDef: {{node Sum}}; Op<name=Sum; signature=input:T, reduction_indices:Tidx -> output:T; attr=keep_dims:bool,default=false; attr=T:type,allowed=[DT_FLOAT, DT_DOUBLE, DT_INT32, DT_UINT8, DT_INT16, ..., DT_UINT16, DT_COMPLEX128, DT_HALF, DT_UINT32, DT_UINT64]; attr=Tidx:type,default=DT_INT32,allowed=[DT_INT32, DT_INT64]> [Op:Sum]

In [61]:
train_ds

<PrefetchDataset shapes: ((None, 180, 180, 3), (None,)), types: (tf.float32, tf.int32)>

In [60]:
model_2.fit(
  train_ds,
  validation_data=val_ds,
  epochs=6
)

Epoch 1/6


TypeError: 'NoneType' object is not callable

In [31]:
model2.summary()

NameError: name 'model2' is not defined

In [17]:
!pwd

/c/ShankersDocs/EDUCATION/RICE_Bootcamp_DataAnalytics/FinalProject_Img_Recognition_Flowers/Final_RICEproject_ImageRecognition_flowers


In [18]:
import sys
print(sys.executable)a

C:\ProgramData\Anaconda3\envs\Mlearning\python.exe


In [19]:
!conda info


     active environment : Mlearning
    active env location : C:\ProgramData\Anaconda3\envs\Mlearning
            shell level : 2
       user config file : C:\Users\nairs\.condarc
 populated config files : C:\Users\nairs\.condarc
          conda version : 4.8.3
    conda-build version : 3.18.11
         python version : 3.7.6.final.0
       virtual packages : __cuda=10.2
       base environment : C:\ProgramData\Anaconda3  (writable)
           channel URLs : https://repo.anaconda.com/pkgs/main/win-64
                          https://repo.anaconda.com/pkgs/main/noarch
                          https://repo.anaconda.com/pkgs/r/win-64
                          https://repo.anaconda.com/pkgs/r/noarch
                          https://repo.anaconda.com/pkgs/msys2/win-64
                          https://repo.anaconda.com/pkgs/msys2/noarch
          package cache : C:\ProgramData\Anaconda3\pkgs
                          C:\Users\nairs\.conda\pkgs
                          C:\Users\nairs\Ap

In [23]:
model_2.save('C:/Users/nairs/Desktop/model_2')

INFO:tensorflow:Assets written to: C:/Users/nairs/Desktop/model_2\assets
